In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load and preview the dataset
data = pd.read_csv('data.csv')
print(data.head())
print(data.info())

# Step 1: Data Preprocessing
# Convert 'Streams' column to numeric, replacing non-numeric entries with NaN
data['Streams'] = pd.to_numeric(data['Streams'], errors='coerce')

# Drop rows with NaN values in 'Streams' and 'Position'
features = ['Streams', 'Position']  # Include both 'Streams' and 'Position' as features for clustering
data = data.dropna(subset=features)

# Standardize the 'Streams' and 'Position' features for clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data[features])

# Step 2: Apply K-Means Clustering
kmeans = KMeans(n_clusters=5, random_state=42)  # Adjust the number of clusters as needed
data['Cluster'] = kmeans.fit_predict(X_scaled)

# Step 3: Evaluate Clustering
silhouette_avg = silhouette_score(X_scaled, data['Cluster'])
print("Silhouette Score:", silhouette_avg)

# Step 4: Content-Based Filtering Function using 'Genre' and 'Artist'
def recommend_content_based(track_name, data, num_recommendations=5):
    genre = data.loc[data['Track Name'] == track_name, 'Genre'].values[0]
    artist = data.loc[data['Track Name'] == track_name, 'Artist'].values[0]

    # Recommend tracks by the same genre and/or artist
    similar_tracks = data[(data['Genre'] == genre) | (data['Artist'] == artist)].sample(num_recommendations)
    return similar_tracks

# Step 5: Hybrid Recommendation System (placeholder for collaborative filtering integration)
def recommend_hybrid(user_id, num_recommendations=5, user_weight=0.5):
    # Placeholder: Implement collaborative filtering function here if needed
    # For now, this function will use content-based recommendations
    content_recs = recommend_content_based(track_name, data, num_recommendations)
    return content_recs

# Step 6: Real-Time Context-Aware Recommendations (example using clustering)
def recommend_with_context(user_id, context_data, num_recommendations=5):
    if 6 <= context_data['time'] <= 18:
        return recommend_hybrid(user_id, num_recommendations)
    else:
        return data[data['Genre'] == 'calm'].sample(num_recommendations)

# Step 7: Build a Deep Learning Model for Enhanced Recommendations
def build_autoencoder(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(input_dim, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 8: User Feedback Integration
def incorporate_user_feedback(user_id, track_name, feedback, data):
    if feedback == 'like':
        data.loc[(data['user_id'] == user_id) & (data['Track Name'] == track_name), 'Streams'] += 1
    elif feedback == 'dislike':
        data.loc[(data['user_id'] == user_id) & (data['Track Name'] == track_name), 'Streams'] -= 1

# Evaluate Diversity of Recommendations
def evaluate_diversity(recommendations, data):
    unique_genres = data.loc[recommendations.index, 'Genre'].nunique()
    print("Diversity Score (Unique Genres):", unique_genres)

# Example: Recommend Songs for a User with Context
track_name = 'Starboy'  # Replace with an actual track name from your dataset
context_data = {'time': 10}  # Example context for daytime
recommendations = recommend_with_context(user_id=1, context_data=context_data)

# Displaying the recommended songs in a table format
print("Recommended Songs:")
print(recommendations[['Track Name', 'Artist', 'Genre', 'Position']])


<ipython-input-14-1a7be22f6055>:12: DtypeWarning: Columns (3,6,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data.csv')


   Position                               Track Name            Artist  \
0         1                                  Starboy        The Weeknd   
1         2                                   Closer  The Chainsmokers   
2         3                          Let Me Love You          DJ Snake   
3         4  Rockabye (feat. Sean Paul & Anne-Marie)      Clean Bandit   
4         5                                One Dance             Drake   

     Streams  Unnamed: 4            Genre Unnamed: 6 Unnamed: 7 Unnamed: 8  \
0  3135625.0         NaN  ['canadian pop'        NaN        NaN        NaN   
1  3015525.0         NaN           ['pop'        NaN        NaN        NaN   
2  2545384.0         NaN           ['pop'        NaN        NaN        NaN   
3  2356604.0         NaN           ['pop'        NaN     'edm']        NaN   
4  2259887.0         NaN   ['toronto rap'        NaN        NaN        NaN   

  Unnamed: 9 Unnamed: 10 Unnamed: 11 Unnamed: 12 Unnamed: 13 Unnamed: 14  \
0        N